In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Define the root directory of your dataset
dataset_root = "MMU-Iris-Database"

# Initialize empty lists to store images and labels
iris_images = []
labels = []

# Iterate through each person's directory
for person_id, person_dir in enumerate(sorted(os.listdir(dataset_root))):
    person_path = os.path.join(dataset_root, person_dir)

    # Skip any non-directory files
    if not os.path.isdir(person_path):
        continue

    # Iterate through "left" and "right" iris images
    for eye_side in ["left", "right"]:
        eye_side_path = os.path.join(person_path, eye_side)
        for image_filename in os.listdir(eye_side_path):
            image_path = os.path.join(eye_side_path, image_filename)

            # Read and preprocess the image (normalize pixel values)
            iris_image = cv2.imread(image_path)

            if iris_image is not None:
                iris_image = iris_image / 255.0  # Normalize pixel values (0-255) to (0-1)

                # Append the preprocessed image and its label to the lists
                iris_images.append(iris_image)
                labels.append(person_id)

# Convert the lists to NumPy arrays
iris_images = np.array(iris_images)
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(iris_images, labels, test_size=0.2, random_state=42)

# Define a more complex CNN model
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=iris_images.shape[1:]))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(layers.Dense(len(np.unique(labels)), activation='softmax'))

# Learning rate scheduling
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=len(X_train) // 32,
    decay_rate=0.9,
    staircase=True
)

# Compile the model with regularization, normalization, and learning rate scheduling
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model with data augmentation
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

model.fit(datagen.flow(X_train, y_train, batch_size= 8), epochs=20, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

# Save the model for future use
model.save('iris_recognition_model.h5')
